In [11]:
import pathlib
import tempfile

import geopandas
import pystac
import rioxarray
import rioxarray.merge
import stac2dcache

from shapely.geometry import Polygon
from stac2dcache.utils import get_asset
from eratosthenes.generic.handler_www import get_tar_file

In [2]:
catalog_url = ("https://webdav.grid.surfsara.nl:2880"
               "/pnfs/grid.sara.nl/data/eratosthenes/"
               "disk/red-glacier_sentinel-2")
collection_id = "sentinel-s2-l1c"
dem_index_url = (
    "https://webdav.grid.surfsara.nl:2880"
    "/pnfs/grid.sara.nl/data/eratosthenes/"
    "disk/GIS/Elevation/ArcticDEM_Tile_Index_Rel7_10m.geojson"
)
tmp_dir = "./"

In [3]:
# configure connection to dCache
dcache = stac2dcache.configure(
    filesystem="dcache", 
    token_filename="macaroon.dat"
)

In [4]:
# read DEM index
with dcache.open(dem_index_url) as f:
    dem_index = geopandas.read_file(f)

In [5]:
def read_catalog(url):
    """
    Read STAC catalog from URL
    
    :param url: urlpath to the catalog root
    :return: PyStac Catalog object
    """
    url = url if url.endswith("catalog.json") else f"{url}/catalog.json"
    catalog = pystac.Catalog.from_file(url)
    return catalog

In [6]:
# read image catalog
catalog = read_catalog(catalog_url)
subcatalog = catalog.get_child(collection_id)

In [7]:
# TODO: loop over catalog, look for all the tiles presents and make dictionary
tiles = []
KEYS = [
    "sentinel:utm_zone", 
    "sentinel:latitude_band", 
    "sentinel:grid_square"
] 

####
# for item in subcatalog.get_all_items():
####
items = subcatalog.get_all_items()
item = next(items)
####

tile_id = "".join([str(item.properties[key]) for key in KEYS])
tiles.append(
    dict(id=tile_id, item=item)
)
# select item with largest data_coverage
item.properties['sentinel:data_coverage']

20.11

In [8]:
# TODO: loop over tiles

###
# for tile in tiles:
###
tile = tiles[0]
###

item = tile['item']

da = get_asset(
    catalog,
    asset_key="B02",
    item_id=item.id,
    filesystem=dcache,
    load=False
)

GDAL headers saved to: /var/folders/t6/r2gjczqj7bb8798wr4g1p87m0000gn/T/tmpx2x9vc2k


In [9]:
bbox = da.rio.bounds()
crs = da.spatial_ref.crs_wkt
transform = da.rio.transform()

tile_geometry = Polygon.from_bounds(*bbox)
dem_index_reproj = dem_index.to_crs(crs)

# select DEM tiles intersecting image bbox
mask = dem_index_reproj.intersects(tile_geometry)
dem_index_reproj = dem_index_reproj[mask]
dem_index_reproj = dem_index_reproj

dem_index_reproj

,tile,fileurl,geometry
1453,51_08,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((536306.646 6623282.652, 551048.710 6..."
1454,51_09,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((506649.406 6714838.166, 521506.591 6..."
1455,51_10,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((476767.162 6806702.367, 491735.967 6..."
1495,52_08,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((445022.334 6593582.205, 459923.633 6..."
1496,52_09,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((415050.354 6684911.721, 430064.285 6..."
1497,52_10,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((384858.403 6776557.020, 399981.428 6..."


In [13]:
# for url in dem_index_reproj.fileurl:
#     get_tar_file(url, "./DEM")

In [12]:
with tempfile.TemporaryDirectory(dir=tmp_dir) as tmpdir:
    
    for url in dem_index_reproj.fileurl:
        get_tar_file(url, tmpdir)

    dem_tiles_filename = pathlib.Path(tmpdir).glob("*_dem.tif")
    dem_tiles = [rioxarray.open_rasterio(f) for f in dem_tiles_filename]

    # merge DEM tiles
    dem = rioxarray.merge.merge_arrays(dem_tiles)

    # reproject to image CRS
    dem_reproj = dem.rio.reproject(crs, transform=transform)

    # crop area within tile
    dem_clip = dem_reproj.rio.clip_box(*bbox)
    
    # save raster file
    output_path = f"{tile['id']}.tif"
    dem_clip.rio.to_raster(output_path)